<a href="https://colab.research.google.com/github/Sushant6862/Jeremy-Howard-Squadron-/blob/main/NYC_TLC_Trip_Duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xgboost lightgbm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.4 MB/s eta 0:00:00


In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
from numpy import math
#from haversine import haversine
import xgboost
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import seaborn as sns
from datetime import datetime
#import klib


from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder


import warnings
warnings.filterwarnings("ignore")
import warnings
from pylab import rcParams
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
warnings.filterwarnings('ignore')

<ipython-input-2-9f99b4999f63>:4: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  from numpy import math


In [ ]:
nyc_tlc = pd.read_parquet('/content/drive/MyDrive/DataSets/yellow_tripdata_2023-11.parquet',engine='pyarrow')

In [ ]:
nyc_tlc

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-11-01 00:03:03,2023-11-01 01:04:08,2.0,13.60,1.0,N,132,26,2,61.80,2.75,0.5,0.00,0.00,1.0,66.05,0.0,1.75
1,1,2023-11-01 00:03:28,2023-11-01 00:23:59,0.0,3.50,1.0,N,140,7,1,20.50,3.50,0.5,5.10,0.00,1.0,30.60,2.5,0.00
2,2,2023-10-31 23:58:05,2023-11-01 00:54:03,4.0,18.61,2.0,N,132,230,1,70.00,0.00,0.5,16.54,6.94,1.0,99.23,2.5,1.75
3,2,2023-11-01 00:03:50,2023-11-01 00:04:59,1.0,0.39,1.0,N,236,236,1,4.40,1.00,0.5,1.88,0.00,1.0,11.28,2.5,0.00
4,2,2023-11-01 00:06:30,2023-11-01 00:14:25,1.0,1.20,1.0,N,236,141,1,10.00,1.00,0.5,3.00,0.00,1.0,18.00,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339710,2,2023-11-30 23:39:21,2023-12-01 00:05:10,NaN,4.15,NaN,None,161,146,0,24.30,0.00,0.5,5.66,0.00,1.0,33.96,NaN,NaN
3339711,2,2023-11-30 23:01:55,2023-11-30 23:03:06,NaN,0.00,NaN,None,232,232,0,22.42,0.00,0.5,5.14,0.00,1.0,31.56,NaN,NaN
3339712,2,2023-11-30 23:23:16,2023-11-30 23:33:56,NaN,1.74,NaN,None,113,186,0,16.09,0.00,0.5,0.00,0.00,1.0,20.09,NaN,NaN
3339713,2,2023-11-30 23:39:22,2023-11-30 23:53:33,NaN,2.39,NaN,None,90,144,0,16.23,0.00,0.5,0.00,0.00,1.0,20.23,NaN,NaN


In [ ]:
nyc_tlc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339715 entries, 0 to 3339714
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [ ]:
traindata = nyc_tlc
null_counts = traindata.isnull().sum()
columns_with_null = null_counts[null_counts > 0]
dtypes_of_columns_with_null = traindata[columns_with_null.index].dtypes

# Create a DataFrame with two columns
df_info = pd.DataFrame({'Null_Counts': null_counts[columns_with_null.index], 'Data_Types': dtypes_of_columns_with_null})

# Display the DataFrame
print("\nCombined Information about Columns with Null Values:")
print(df_info)


Combined Information about Columns with Null Values:
                      Null_Counts Data_Types
passenger_count            132675    float64
RatecodeID                 132675    float64
store_and_fwd_flag         132675     object
congestion_surcharge       132675    float64
Airport_fee                132675    float64


In [ ]:
nyc_tlc.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [ ]:
# #Calculate trip duration from pickup time to dropout time.
# nyc_tlc['trip_duration'] = (nyc_tlc['tpep_dropoff_datetime'] - nyc_tlc['tpep_pickup_datetime']).dt.total_seconds()
# columns_to_drop = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','VendorID','store_and_fwd_flag','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge']
# nyc_df = nyc_tlc.drop(columns=columns_to_drop)

In [ ]:
#Calculate trip duration from pickup time to dropout time.
nyc_tlc['trip_duration'] = (nyc_tlc['tpep_dropoff_datetime'] - nyc_tlc['tpep_pickup_datetime']).dt.total_seconds()
columns_to_drop = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','VendorID','PULocationID','store_and_fwd_flag','congestion_surcharge','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge']
nyc_df = nyc_tlc.drop(columns=columns_to_drop)

In [ ]:
nyc_df.head()

,passenger_count,trip_distance,RatecodeID,DOLocationID,payment_type,total_amount,Airport_fee,trip_duration
0,2.0,13.60,1.0,26,2,66.05,1.75,3665.0
1,0.0,3.50,1.0,7,1,30.60,0.00,1231.0
2,4.0,18.61,2.0,230,1,99.23,1.75,3358.0
3,1.0,0.39,1.0,236,1,11.28,0.00,69.0
4,1.0,1.20,1.0,141,1,18.00,0.00,475.0


In [ ]:
nyc_df=nyc_df.dropna()

In [ ]:
traindata = nyc_df
null_counts = traindata.isnull().sum()
columns_with_null = null_counts[null_counts > 0]
dtypes_of_columns_with_null = traindata[columns_with_null.index].dtypes

# Create a DataFrame with two columns
df_info = pd.DataFrame({'Null_Counts': null_counts[columns_with_null.index], 'Data_Types': dtypes_of_columns_with_null})

# Display the DataFrame
print("\nCombined Information about Columns with Null Values:")
print(df_info)


Combined Information about Columns with Null Values:
Empty DataFrame
Columns: [Null_Counts, Data_Types]
Index: []


In [ ]:
# label_encoder = LabelEncoder()

# nyc_df['store_and_fwd_flag'] = label_encoder.fit_transform(nyc_df['store_and_fwd_flag'])

In [ ]:
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3207040 entries, 0 to 3207039
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   passenger_count  float64
 1   trip_distance    float64
 2   RatecodeID       float64
 3   DOLocationID     int32  
 4   payment_type     int64  
 5   total_amount     float64
 6   Airport_fee      float64
 7   trip_duration    float64
dtypes: float64(6), int32(1), int64(1)
memory usage: 208.0 MB


In [ ]:
nyc_df_float = nyc_df.astype(float)

In [ ]:
# nyc_df_float.corr()

In [ ]:

# fig = plt.figure(figsize = (18,18))

# fig.add_subplot(1,1,1)
# sns.heatmap(nyc_df_float.corr(), annot = True)    # annot = True, to print the values of the Correlation Coefficients.

# plt.show()

In [ ]:
# columns_to_drop = ['extra', 'mta_tax','total_amount','VendorID','tolls_amount','tpep_pickup_datetime','tpep_dropoff_datetime','tip_amount','airport_fee','store_and_fwd_flag']
# df_taxi_1 = Df_NYC.drop(columns=columns_to_drop)

In [ ]:
nyc_df_float.shape

(3207040, 8)

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming 'nyc_df' is your DataFrame
# Replace this with your actual data or load your dataset

# Specify the variables for which to calculate VIF
# Exclude variables that might be dependent or cause multicollinearity
exclude_vars = ['trip_duration']  # Add other variables to exclude if necessary
x_vars = [col for col in nyc_df_float.columns if col not in exclude_vars]

# Create a DataFrame containing only the specified x_vars
x_data = nyc_df_float[x_vars]

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["Variable"] = x_vars
vif_data["VIF"] = [variance_inflation_factor(x_data.values, i) for i in range(x_data.shape[1])]

# Display the VIF values
print(vif_data)


          Variable       VIF
0  passenger_count  3.068424
1    trip_distance  1.590433
2       RatecodeID  1.051372
3     DOLocationID  4.312133
4     payment_type  4.063623
5     total_amount  3.898675
6      Airport_fee  1.743840


In [ ]:
nyc_df_float=nyc_df_float.drop(["DOLocationID"],axis=1)

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming 'nyc_df' is your DataFrame
# Replace this with your actual data or load your dataset

# Specify the variables for which to calculate VIF
# Exclude variables that might be dependent or cause multicollinearity
exclude_vars = ['trip_duration']  # Add other variables to exclude if necessary
x_vars = [col for col in nyc_df_float.columns if col not in exclude_vars]

# Create a DataFrame containing only the specified x_vars
x_data = nyc_df_float[x_vars]

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["Variable"] = x_vars
vif_data["VIF"] = [variance_inflation_factor(x_data.values, i) for i in range(x_data.shape[1])]

# Display the VIF values
print(vif_data)


          Variable       VIF
0  passenger_count  2.855366
1    trip_distance  1.581950
2       RatecodeID  1.051213
3     payment_type  2.851939
4     total_amount  3.587792
5      Airport_fee  1.710177


In [ ]:
# nyc_df_float=nyc_df_float.drop(["PULocationID"],axis=1)

In [ ]:
# import pandas as pd
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # Assuming 'nyc_df' is your DataFrame
# # Replace this with your actual data or load your dataset

# # Specify the variables for which to calculate VIF
# # Exclude variables that might be dependent or cause multicollinearity
# exclude_vars = ['trip_duration']  # Add other variables to exclude if necessary
# x_vars = [col for col in nyc_df_float.columns if col not in exclude_vars]

# # Create a DataFrame containing only the specified x_vars
# x_data = nyc_df_float[x_vars]

# # Calculate VIF for each variable
# vif_data = pd.DataFrame()
# vif_data["Variable"] = x_vars
# vif_data["VIF"] = [variance_inflation_factor(x_data.values, i) for i in range(x_data.shape[1])]

# # Display the VIF values
# print(vif_data)


In [ ]:

# Calculate Q1, Q3, and IQR for each column
Q1 = nyc_df_float.quantile(0.05)
Q3 = nyc_df_float.quantile(0.95)
IQR = Q3 - Q1

# Define lower and upper bounds for each column
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers for each column
outliers = (nyc_df_float < lower_bound) | (nyc_df_float > upper_bound)

# Optionally, remove outliers from the DataFrame
filtered_dataframe1 = nyc_df_float[~outliers.any(axis=1)]

# Display information about outliers and filtered DataFrame
print("Columns with Outliers:")
print(outliers.sum())

print("\nOriginal DataFrame Shape:", nyc_df_float.shape)
print("Filtered DataFrame Shape:", filtered_dataframe1.shape)

Columns with Outliers:
passenger_count       55
trip_distance       1438
RatecodeID         49081
payment_type       48425
total_amount        3012
Airport_fee            0
trip_duration       5086
dtype: int64

Original DataFrame Shape: (3207040, 7)
Filtered DataFrame Shape: (3106185, 7)


In [ ]:
nyc_df_1 = filtered_dataframe1

In [ ]:
X=nyc_df_1.drop(columns=["trip_duration"])
y=nyc_df_1["trip_duration"]

In [ ]:
y

0          3665.0
1          1231.0
2          3358.0
3            69.0
4           475.0
            ...  
3207035    1170.0
3207036     905.0
3207037     753.0
3207038     372.0
3207039     325.0
Name: trip_duration, Length: 3106185, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create the Decision Tree Regressor
decision_tree = DecisionTreeRegressor()

# Train the model on the training set
decision_tree.fit(X_train, y_train)

y_pred_test_DT = decision_tree.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_test_DT)
print(f"Mean Squared Error: {mse}")

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred_test_DT)
print(f"Mean Absolute Error: {mae}")

# Calculate Root Mean Squared Logarithmic Error
rmsle = np.sqrt(np.mean(np.log1p(y_pred_test_DT) - np.log1p(y_test))**2)
print(f"Root Mean Squared Logarithmic Error: {rmsle}")

# Calculate R-squared
r2_DT = r2_score(y_test, y_pred_test_DT)
print(f"R-squared: {r2_DT}")

# Calculate Adjusted R-squared
n = len(X_test)
p = X_test.shape[1]  # number of features
adjusted_r2 = 1 - (1 - r2_DT) * (n - 1) / (n - p - 1)
print(f"Adjusted R-squared: {adjusted_r2}")




Mean Squared Error: 118657.0288013889
Root Mean Squared Error: 344.4662955956488
Mean Absolute Error: 144.13894900059307
Root Mean Squared Logarithmic Error: 0.012348788245836911
R-squared: 0.8126244172045232
Adjusted R-squared: 0.8126226074826862
Mean Absolute Percentage Error: inf


In [ ]:
# Calculate Mean Absolute Percentage Error
mape = np.mean(np.abs((y_test - y_pred_test_DT) / np.maximum(np.abs(y_test), 1))) * 100
print(f"Mean Absolute Percentage Error: {mape}")


Mean Absolute Percentage Error: 33.08449938629339


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create the Linear Regression model
linear_reg = LinearRegression()

# Train the model on the training set
linear_reg.fit(X_train, y_train)

y_pred_test_LIR = linear_reg.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_test_LIR)
print(f"Mean Squared Error: {mse}")

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred_test_LIR)
print(f"Mean Absolute Error: {mae}")

# Calculate Root Mean Squared Logarithmic Error
rmsle = np.sqrt(np.mean(np.log1p(y_pred_test_LIR) - np.log1p(y_test))**2)
print(f"Root Mean Squared Logarithmic Error: {rmsle}")

# Calculate R-squared
r2_LIR = r2_score(y_test, y_pred_test_LIR)
print(f"R-squared: {r2_LIR}")

# Calculate Adjusted R-squared
n = len(X_test)
p = X_test.shape[1]  # number of features
adjusted_r2 = 1 - (1 - r2_LIR) * (n - 1) / (n - p - 1)
print(f"Adjusted R-squared: {adjusted_r2}")

# Calculate Mean Absolute Percentage Error
mape = np.mean(np.abs((y_test - y_pred_test_LIR) / np.maximum(np.abs(y_test), 1))) * 100
print(f"Mean Absolute Percentage Error: {mape}")


Mean Squared Error: 175456.52309560325
Root Mean Squared Error: 418.8753073357312
Mean Absolute Error: 260.75593211980635
Root Mean Squared Logarithmic Error: 0.1240562971155228
R-squared: 0.7229302924369039
Adjusted R-squared: 0.7229276164260137
Mean Absolute Percentage Error: 97.7234610218689


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create the RandomForestRegressor model
rf_regressor = RandomForestRegressor(n_estimators=51)
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_regressor.predict(X_test)

# Calculate Mean Squared Error
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Mean Squared Error: {mse_rf}")

# Calculate Root Mean Squared Error
rmse_rf = np.sqrt(mse_rf)
print(f"Root Mean Squared Error: {rmse_rf}")

# Calculate Mean Absolute Error
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Mean Absolute Error: {mae_rf}")

# Calculate Root Mean Squared Logarithmic Error
rmsle_rf = np.sqrt(np.mean(np.log1p(y_pred_rf) - np.log1p(y_test))**2)
print(f"Root Mean Squared Logarithmic Error: {rmsle_rf}")

# Calculate R-squared
r2_rf = r2_score(y_test, y_pred_rf)
print(f"R-squared: {r2_rf}")

# Calculate Adjusted R-squared (Not applicable for RandomForestRegressor as it does not involve multiple features)
# Adjusted R-squared is more suitable for models with multiple features.

# Calculate Mean Absolute Percentage Error
mape_rf = np.mean(np.abs((y_test - y_pred_rf) / np.maximum(np.abs(y_test), 1))) * 100
print(f"Mean Absolute Percentage Error: {mape_rf}")

# Optionally, you can also inspect feature importances
feature_importances_rf = rf_regressor.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X.columns, feature_importances_rf):
    print(f"{feature}: {importance}")


Mean Squared Error: 86565.37427078202
Root Mean Squared Error: 294.21994200050756
Mean Absolute Error: 129.1712459514094
Root Mean Squared Logarithmic Error: 0.017868801301058477
R-squared: 0.8633015033517635
Mean Absolute Percentage Error: 32.37314724064272
Feature Importances:
passenger_count: 0.009467244690288168
trip_distance: 0.08734912561545159
RatecodeID: 0.011013740133255213
payment_type: 0.015121032617461337
total_amount: 0.8713331103377925
Airport_fee: 0.005715746605751163


Mean Squared Error: 86565.37427078202

Root Mean Squared Error: 294.21994200050756

Mean Absolute Error: 129.1712459514094

Root Mean Squared Logarithmic Error: 0.017868801301058477

R-squared: 0.8633015033517635

Mean Absolute Percentage Error: 32.37314724064272

Feature Importances:

passenger_count: 0.009467244690288168

trip_distance: 0.08734912561545159

RatecodeID: 0.011013740133255213

payment_type: 0.015121032617461337

total_amount: 0.8713331103377925

Airport_fee: 0.005715746605751163


1716PM

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create and train the XGBoost regression model
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42,n_estimators=48,)
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_regressor.predict(X_test)

# Calculate Mean Squared Error
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f"XGBoost Mean Squared Error: {mse_xgb}")

# Calculate Root Mean Squared Error
rmse_xgb = np.sqrt(mse_xgb)
print(f"XGBoost Root Mean Squared Error: {rmse_xgb}")

# Calculate Mean Absolute Error
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost Mean Absolute Error: {mae_xgb}")

# Calculate Root Mean Squared Logarithmic Error
rmsle_xgb = np.sqrt(np.mean(np.log1p(y_pred_xgb) - np.log1p(y_test))**2)
print(f"XGBoost Root Mean Squared Logarithmic Error: {rmsle_xgb}")

# Calculate R-squared
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost R-squared: {r2_xgb}")

# Calculate Adjusted R-squared (Not applicable for XGBoostRegressor as it does not involve multiple features)
# Adjusted R-squared is more suitable for models with multiple features.

# Calculate Mean Absolute Percentage Error
mape_xgb = np.mean(np.abs((y_test - y_pred_xgb) / np.maximum(np.abs(y_test), 1))) * 100
print(f"XGBoost Mean Absolute Percentage Error: {mape_xgb}")


XGBoost Mean Squared Error: 86954.20596316845
XGBoost Root Mean Squared Error: 294.879985694466
XGBoost Mean Absolute Error: 158.23909121324917
XGBoost Root Mean Squared Logarithmic Error: 0.03775203526798858
XGBoost R-squared: 0.8626874852383299
XGBoost Mean Absolute Percentage Error: 41.12376395367203


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create and train the LightGBM regression model
lgb_regressor = lgb.LGBMRegressor()
lgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred_lgb = lgb_regressor.predict(X_test)

# Calculate Mean Squared Error
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
print(f"LightGBM Mean Squared Error: {mse_lgb}")

# Calculate Root Mean Squared Error
rmse_lgb = np.sqrt(mse_lgb)
print(f"LightGBM Root Mean Squared Error: {rmse_lgb}")

# Calculate Mean Absolute Error
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
print(f"LightGBM Mean Absolute Error: {mae_lgb}")

# Calculate Root Mean Squared Logarithmic Error
rmsle_lgb = np.sqrt(np.mean(np.log1p(y_pred_lgb) - np.log1p(y_test))**2)
print(f"LightGBM Root Mean Squared Logarithmic Error: {rmsle_lgb}")

# Calculate R-squared
r2_lgb = r2_score(y_test, y_pred_lgb)
print(f"LightGBM R-squared: {r2_lgb}")

# Calculate Adjusted R-squared (Not applicable for LightGBMRegressor as it does not involve multiple features)
# Adjusted R-squared is more suitable for models with multiple features.

# Calculate Mean Absolute Percentage Error
mape_lgb = np.mean(np.abs((y_test - y_pred_lgb) / np.maximum(np.abs(y_test), 1))) * 100
print(f"LightGBM Mean Absolute Percentage Error: {mape_lgb}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.306168 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 724
[LightGBM] [Info] Number of data points in the train set: 2484948, number of used features: 7
[LightGBM] [Info] Start training from score 1010.336710
LightGBM Mean Squared Error: 84924.74319971872
LightGBM Root Mean Squared Error: 291.4185018143473
LightGBM Mean Absolute Error: 155.69829574498954
LightGBM Root Mean Squared Logarithmic Error: 0.036410406003747633
LightGBM R-squared: 0.865892282896795
LightGBM Mean Absolute Percentage Error: 41.00610899489028


In [ ]:
import catboost
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create and train the CatBoost regression model
catboost_regressor = catboost.CatBoostRegressor(random_state=42)
catboost_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred_catboost = catboost_regressor.predict(X_test)

# Calculate Mean Squared Error
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f"CatBoost Mean Squared Error: {mse_catboost}")

# Calculate Root Mean Squared Error
rmse_catboost = np.sqrt(mse_catboost)
print(f"CatBoost Root Mean Squared Error: {rmse_catboost}")

# Calculate Mean Absolute Error
mae_catboost = mean_absolute_error(y_test, y_pred_catboost)
print(f"CatBoost Mean Absolute Error: {mae_catboost}")

# Calculate Root Mean Squared Logarithmic Error
rmsle_catboost = np.sqrt(np.mean(np.log1p(y_pred_catboost) - np.log1p(y_test))**2)
print(f"CatBoost Root Mean Squared Logarithmic Error: {rmsle_catboost}")

# Calculate R-squared
r2_catboost = r2_score(y_test, y_pred_catboost)
print(f"CatBoost R-squared: {r2_catboost}")

# Calculate Adjusted R-squared (Not applicable for CatBoostRegressor as it does not involve multiple features)
# Adjusted R-squared is more suitable for models with multiple features.

# Calculate Mean Absolute Percentage Error
mape_catboost = np.mean(np.abs((y_test - y_pred_catboost) / np.maximum(np.abs(y_test), 1))) * 100
print(f"CatBoost Mean Absolute Percentage Error: {mape_catboost}")


Learning rate set to 0.140812
0:	learn: 712.2689043	total: 570ms	remaining: 9m 29s
1:	learn: 642.2600850	total: 908ms	remaining: 7m 32s
2:	learn: 583.1126294	total: 1.24s	remaining: 6m 52s
3:	learn: 534.7831836	total: 1.55s	remaining: 6m 26s
4:	learn: 494.9156031	total: 1.88s	remaining: 6m 14s
5:	learn: 462.2799199	total: 2.2s	remaining: 6m 5s
6:	learn: 436.2326476	total: 2.53s	remaining: 5m 58s
7:	learn: 415.5367038	total: 2.84s	remaining: 5m 52s
8:	learn: 399.3601349	total: 3.17s	remaining: 5m 48s
9:	learn: 385.7551818	total: 3.47s	remaining: 5m 43s
10:	learn: 375.1642939	total: 3.78s	remaining: 5m 39s
11:	learn: 366.3801030	total: 4.1s	remaining: 5m 37s
12:	learn: 359.4073774	total: 4.41s	remaining: 5m 34s
13:	learn: 352.8485011	total: 4.72s	remaining: 5m 32s
14:	learn: 348.2313609	total: 5.03s	remaining: 5m 30s
15:	learn: 344.1249682	total: 5.33s	remaining: 5m 27s
16:	learn: 340.3236962	total: 5.63s	remaining: 5m 25s
17:	learn: 337.1903056	total: 5.94s	remaining: 5m 23s
18:	learn: 